## Data Exploration
### insights
- charges on the bank account (dkb) appear in the paypal statements as pending and with type is bank deposit


### next steps:
1. [x] we dropped the wrong data from the paypal, get better match by looking for the positve entries in the paypal data, the one corresponding to the deposit are status=pending, that's why they have been dropped -> rewrite `get_paypal_index_for_single_row` to work with the raw paypal data and drop only later
1. [ ] drop paypal from dkb data without asignment to paypal and run
1. [ ] drop non relevant data from paypal without asignment to dkb and run
1. [ ] run models on paypal and dkb independently and run analysis
1. [ ] later try to match paypal and dkb data

todos:
1. save data for each account -> add_data(filename, account)
2. to each account, find the paypal transactions and the don't consider them in the training, instead classify the remaining paypal data entries
3.  merge paypal and dkb to one dataset
<!--  merge with baysian model -->
 

In [1]:
# ! python -m textblob.download_corpora

In [2]:
# ! pip install pandas textblob colorama tabulate

In [1]:
# WRKDIR = '/home/jovyan/work/'
# import sys

# sys.path.insert(0,WRKDIR)


%load_ext autoreload

%autoreload 2

In [2]:
import numpy as np
import pandas as pd
from datetime import datetime, timezone
import re
from pathlib import Path
from collections import Counter

### define functions

In [3]:
# def read_dkb_csv(filename, drop_duplicates=True, verbose=1)-> pd.DataFrame:
#     """Read a file in the CSV format that dkb provides downloads in.

#     Returns a pd.DataFrame with columns of 'date', 'desc', and 'amount'."""
#     account_nr = re.search('\d{10}', Path(filename).name)[0]
#     data=pd.read_csv(filename,sep=';', skiprows=9, encoding='latin-1',
#                      usecols=[0,2,3,4,7,9,10],
#                      names=['Buchungstag', 'Wertstellung', 'Buchungstext','Auftraggeber / Beguenstigter', 'Verwendungszweck', 'Kontonummer',
#                            'BLZ', 'Betrag (EUR)', 'Glaeubiger-ID', 'Mandatsreferenz','Kundenreferenz', 'Unnamed'] )

#     if drop_duplicates:
#         if verbose > 1:
#             duplicated_data = data[data.duplicated()]
#             print(f"dropping {len(duplicated_data)} duplicated entries")
#             if verbose > 2:
#                 if len(duplicated_data)>0:
#                     print(duplicated_data)
#                     print('\n')
#         data.drop_duplicates(inplace=True)
        
    
#     data = data[data['Verwendungszweck'] != 'Tagessaldo'] # drop all the Tagessaldo entries
    
#     print(f"number of data loaded: {len(data)}")
    
#     data['Buchungstag'] = pd.to_datetime(data['Buchungstag'], format = "%d.%m.%Y")
#     # data['Wertstellung'] = pd.to_datetime(data['Wertstellung'], format = "%d.%m.%Y")
#     data['account_nr'] = account_nr
#     if verbose > 1:
#         print(f"new data from {filename} with {len(data)} entries, years {data['Buchungstag'].dt.year.unique()}")


#     data.fillna("", inplace=True)
#     # if drop_duplicates:
#     #     if verbose > 1:
#     #         duplicated_data = data[data.duplicated()]
#     #         print(f"dropping {len(duplicated_data)} duplicated entries")
#     #         # if len(duplicated_data)>0:
#     #         #     print(duplicated_data)
#     #         #     print('\n')
#     #     data.drop_duplicates(inplace=True)
#     # print(len(data))

#     data['date'] = data['Buchungstag'] #.dt.strftime('%d/%m/%Y')
#     data['amount'] = data['Betrag (EUR)'].str.replace('.', '', regex=False).str.replace(',', '.', regex=False)
#     data.drop(data[data['amount']==''].index, inplace=True)
#     data['amount'] = data['amount'].astype(float)

#     data = data[data['amount'].astype(float)!=0] # drop all zero values

#     desc_column_names = ['Buchungstext', 'Auftraggeber / Beguenstigter', 'Verwendungszweck']
#     data['desc'] = data[desc_column_names].agg(' '.join, axis=1)
#     df = data[['date', 'amount', 'desc', 'account_nr']]
#     # df = data


#     return df

In [4]:


def read_paypal_csv(filename,verbose=1)-> pd.DataFrame:
    """Read a file in the CSV format that paypal provides downloads in.

    Returns a pd.DataFrame with columns of 'datetime', 'description', and 'amount'."""

    data = pd.read_csv(filename, decimal=",", dtype="string")

    if verbose>1:
        print(f"loaded {len(data)} paypal records")
        
    data = data[data['Balance Impact']!='Memo']
    if verbose>1:
        print(f"{len(data)} paypal records after dropping `Balance Impact` = `Memo` entries")        
        
    # data = data[data['Status']!='Pending']
    # if verbose>1:
    #     print(f"{len(data)} paypal records after dropping `Status` = `Pending` entries")     

    type_list = [
        'Bank Deposit to PP Account',
        'Payment Refund`'
        ]

    #tmp comment out
    # data = data[data['Type'].str.contains('|'.join(type_list))]
    # # data = data[data['Status'] == 'Completed']
    # if verbose>1:
    #     print(f"{len(data)} status completed")


    data['TimeZone'].replace('CEST', '+02:00', inplace=True)
    data['TimeZone'].replace('CET', '+01:00', inplace=True)

    data.insert(0, 'datetime', pd.to_datetime(data['Date'] + " " + data['Time']+ data['TimeZone'], format = "%d/%m/%Y %H:%M:%S%z", utc=True))
    # data.drop(['Time', 'Date', 'TimeZone', 'Status'], axis=1, inplace=True) #tmp comment out


    # to make the transaction ID unique we add the datetime
    data['Transaction ID'] = data['Transaction ID'] + "-" +data['datetime'].dt.strftime("%d%m%Y %H:%M:%S%z")

    data.set_index('Transaction ID', inplace=True)

    # convert to floats
    for k in ['Balance', 'Fee', 'Net', 'Gross']:
        data[k] = data[k].str.replace('.','', regex=False).str.replace(',','.', regex=False).astype(float)
    # data = data[data['Balance'] != 0]

    # insert new column description
    data.insert(1, 'description', data[['Name', 'Country', 'Subject', 'Note']].fillna('').agg(' '.join, axis=1))

    # data.drop(['Name', 'Country', 'Subject', 'Note'], axis=1, inplace=True) # drop the columns that are in the desciption now


    data.rename(columns={"Gross": "amount"}, inplace=True)


    # data = data[['datetime', 'description', 'Type',  'amount', 'From Email Address', 'To Email Address']]
    # data = data[['datetime', 'Balance Impact', 'Status', 'description', 'Type',  'amount', 'From Email Address', 'To Email Address']]

    return data


In [5]:
def paypal_drop_reimbursed_positions(data, verbose=1):
    indecies_to_drop = []

    for i, r in data[data.duplicated()].iterrows():

        # print(data[data['amount']==-r['amount']])
        # print(data[(data['datetime']==r['datetime']) & (data['amount']==-r['amount'])])
        # if we find duplicated data we check if there was an immediate reimbursement    
        if verbose >=2:
            print(f"found duplicated data: \t{i}\t amount: {r['amount']} ")
        reimbursement = data[(data['datetime']==r['datetime']) & (data['amount']==-r['amount'])]
        if len(reimbursement)==1:
            reimbursement = reimbursement.iloc[0]
            if verbose >=2:
                print(f"\t => reimbursed \t{reimbursement.name}\t amount: {reimbursement['amount']} ")
            indecies_to_drop.append(i)
            indecies_to_drop.append(reimbursement.name)
    # print(indecies_to_drop)
    # print(data.loc[indecies_to_drop])
    assert data.loc[indecies_to_drop]['amount'].sum() == 0, f"sum ({data.loc[indecies_to_drop]['amount'].sum()}) of the reimbursed values should be zero!"
    if verbose >=2:
        print(f'dropping {len(indecies_to_drop)} reimbursement values')
    return data.drop(indecies_to_drop)

In [6]:
def get_paypal_index_for_single_row(row, data_paypal, max_day_diff = 7, verbose=1):
    """
    row: datarow for which to find the paypal index, e.g. entry from dkb
    data_paypal: paypal data in which to look for the entry
    max_day_diff: max difference in days that the timestamp of the row may differ from the entry in the paypal data
    
    """

    if  ('PAYPAL' not in row['desc'].upper()):
        return np.nan, None
    paypal = data_paypal[data_paypal['amount'].abs() == abs(row['amount'])]

    if len(paypal) == 0:
        # if it is a deposit, then we have to invert the amount
        if verbose>0:
            print('suspect deposit looking for inverted amount')
        paypal = data_paypal[data_paypal['amount'] == -row['amount']]
        if len(paypal) == 0:

            if verbose>0:
                print(f"{row['date']} did not find entry with amount |{row['amount']}|")
            return np.nan, None

    record, time_diff =  get_closest_record(paypal, time_index = row['date'].tz_localize(timezone.utc))

    if abs(time_diff.days) > max_day_diff:
        if verbose>0:
            print(f"WARNING: large timediff {time_diff.days} for {len(paypal['datetime'])} records on {str(row['date']).split(' ')[0]} amout {row['amount']}")
            print('***** paypal record ******')
            print(record)
            print('***** bank row ******')
            print(row['date'], row['amount'])
            print(row['desc'])
            print('\n')
            return np.nan, None

    if verbose>2:
        print(f'\n entry (amount {row["amount"]}): {row["desc"]}')
        print('record:', record)
    return record.name, float(time_diff.days)


def get_paypal_index(data, data_paypal, max_day_diff = 7, verbose=1):
    """
    data: data for which to find the paypal index, e.g. entry from dkb
    data_paypal: paypal data in which to look for the entry
    max_day_diff: max difference in days that the timestamp of the row may differ from the entry in the paypal data
    
    returns list of tuples, first is the index in the data, second the  paypal index
    
    """
    indecies = []
    for i, row in data[data['desc'].str.lower().str.contains('paypal')].iterrows():
        index, days = get_paypal_index_for_single_row(row, data_paypal, max_day_diff = max_day_diff, verbose=verbose)
        indecies.append((i,index))
    return indecies

In [7]:

def get_closest_record(records, time_index):
    """
    returns the record, where the datetime index is closest to time_index
    and the min time
    
    """
    min_time = (records['datetime'] - time_index).abs().min()
    
    closest_record = records[(records['datetime'] - time_index).abs() == min_time]
    return closest_record.iloc[0], min_time
        

### final

In [8]:
input_data_path =  Path('/Users/jangie/Documents/Finanzen/Detailed_Data')
categories = {}
with open('../data/categories.txt') as f:
    for i, line in enumerate(f.readlines()):
        categories[i] = line.strip()
        
categories

{0: 'Income - Salary & Benefits',
 1: 'Income - Presents',
 2: 'Income - Reimbursement',
 3: 'Income - Investments',
 4: 'Bill - Utilities, Rent & Fees',
 5: 'Bill - Communications',
 6: 'Bill - Insurrance',
 7: 'Bill - Health',
 8: 'Bill - Others',
 9: 'Holidays - Accomodation',
 10: 'Holidays - Transport',
 11: 'Holidays - Others',
 12: 'Car',
 13: 'Public Transport',
 14: 'Presents for Others',
 15: 'Supermarket & Everyday commodities',
 16: 'Restaurants & Bars',
 17: 'Entertainment, Hobbies & Memberships',
 18: 'Education',
 19: 'Personal Care & Sport',
 20: 'Investments',
 21: 'Other',
 22: 'Out-of-pocket expense',
 23: 'Home Improvement & Stationary',
 24: 'Clothes & Shopping'}

In [9]:
df_dkb = None
drop_duplicates = False

for f in (input_data_path/'dkb').glob('*.csv'):
    assert f.exists()
    # df_dkb = pd.read_csv(str(dkb_data_file))
    df_dkb_in = read_dkb_csv(f, verbose=3, drop_duplicates=drop_duplicates)
    if df_dkb is None:
        df_dkb = df_dkb_in
    else: 
        df_dkb = pd.concat([df_dkb, df_dkb_in], ignore_index=True)
        if drop_duplicates:
            print(f"dropping duplicates considering only ['date', 'amount', 'desc', 'account_nr'] (length {len(df_dkb)})")
            df_dkb.drop_duplicates(subset=['date', 'amount', 'desc', 'account_nr'], inplace=True)
    print(f"=> total length {len(df_dkb)}")
    
df_dkb['target account'] = np.nan

number of data loaded: 232
new data from /Users/jangie/Documents/Finanzen/Detailed_Data/dkb/2020_1071944217.csv with 232 entries, years [2020]
=> total length 230
number of data loaded: 200
new data from /Users/jangie/Documents/Finanzen/Detailed_Data/dkb/2021_1002423109.csv with 200 entries, years [2021]
=> total length 428
number of data loaded: 135
new data from /Users/jangie/Documents/Finanzen/Detailed_Data/dkb/2020_1002423109.csv with 135 entries, years [2020]
=> total length 560
number of data loaded: 589
new data from /Users/jangie/Documents/Finanzen/Detailed_Data/dkb/2021_1071944217.csv with 589 entries, years [2021]
=> total length 1147
number of data loaded: 148
new data from /Users/jangie/Documents/Finanzen/Detailed_Data/dkb/2022_1002423109.csv with 148 entries, years [2022]
=> total length 1294
number of data loaded: 397
new data from /Users/jangie/Documents/Finanzen/Detailed_Data/dkb/2022_1071944217.csv with 397 entries, years [2022]
=> total length 1689


In [10]:
# df_dkb

#### filter out the transaction to other accounts
this is just moving money between accounts and are not real expenses or income. Later we will try to find the corresponding entires to check the consistency. But this is actually not so trivial.

In [13]:
# filter out the paypal entries
paypal_indecies = df_dkb['desc'].str.upper().str.contains('PAYPAL \(EUROPE\) S.A.R.L')
df_dkb.loc[paypal_indecies, 'target account'] = 'paypal'


# filter out the paypal entries
dkb_indecies = df_dkb['desc'].str.contains('Überweisung HERR ')
df_dkb.loc[dkb_indecies, 'target account'] = 'dkb'

credit_indecies = df_dkb['desc'].str.contains('Umbuchung KREDITKARTEN GELDANLAGE')
df_dkb.loc[credit_indecies, 'target account'] = 'credit card'



# df = df_dkb[~paypal_indecies]

In [14]:
df_dkb['account_nr'].unique()

array(['1071944217', '1002423109'], dtype=object)

In [15]:
df_dkb.groupby(['target account', 'account_nr'])['amount'].sum()

target account  account_nr
credit card     1002423109     8992.39
                1071944217    13213.98
dkb             1071944217   -12000.00
paypal          1002423109     -752.94
                1071944217   -22278.67
Name: amount, dtype: float64

In [16]:
df_dkb['target account'].unique()

array(['paypal', nan, 'credit card', 'dkb'], dtype=object)

In [17]:
df_dkb[df_dkb['target account'].isna()]

,date,amount,desc,account_nr,target account
4,2020-12-21,-5.37,Kartenzahlung/-abrechnung REWE SAGT DANKE. 426...,1071944217,NaN
5,2020-12-21,-12.71,Kartenzahlung/-abrechnung DIRK ROSSMANN GMBH//...,1071944217,NaN
6,2020-12-21,-8.71,"Kartenzahlung/-abrechnung DANKE, IHR LIDL//Ber...",1071944217,NaN
7,2020-12-21,-48.20,Kartenzahlung/-abrechnung DM FIL.2814 H:65273/...,1071944217,NaN
8,2020-12-21,-2.80,"Kartenzahlung/-abrechnung EDEKA ZAHL, BERLIN//...",1071944217,NaN
...,...,...,...,...,...
1680,2022-01-21,-17.25,Kartenzahlung/-abrechnung DM FIL.2814 H:65273/...,1071944217,NaN
1681,2022-01-21,-21.95,Kartenzahlung/-abrechnung HUGENDUBEL 0665 FIL....,1071944217,NaN
1682,2022-01-20,-40.96,"Kartenzahlung/-abrechnung EDEKA ZAHL, BERLIN//...",1071944217,NaN
1683,2022-01-20,-8.22,Kartenzahlung/-abrechnung HANSEN OBST GBR GIR ...,1071944217,NaN


In [18]:
!ls ..

BankClassify.py             data_backup
BankClassify_old.py         data_extern
README.md                   envs
Statement_Example.txt       example.py
__pycache__                 notebooks
all_data_dkb_1002423109.csv paypal_data.csv
all_data_dkb_1071944217.csv requirements.txt
categories_old.txt          spec-file.txt
classify.py                 test
data


In [19]:
df_dkb

,date,amount,desc,account_nr,target account
0,2020-12-29,-8.00,"Lastschrift PAYPAL (EUROPE) S.A.R.L ET CIE,S.C...",1071944217,paypal
1,2020-12-22,-9.98,"Lastschrift PAYPAL (EUROPE) S.A.R.L ET CIE,S.C...",1071944217,paypal
2,2020-12-22,-35.84,"Lastschrift PAYPAL (EUROPE) S.A.R.L ET CIE,S.C...",1071944217,paypal
3,2020-12-22,-33.39,"Lastschrift PAYPAL (EUROPE) S.A.R.L ET CIE,S.C...",1071944217,paypal
4,2020-12-21,-5.37,Kartenzahlung/-abrechnung REWE SAGT DANKE. 426...,1071944217,NaN
...,...,...,...,...,...
1684,2022-01-18,-35.70,"Lastschrift PAYPAL (EUROPE) S.A.R.L ET CIE,S.C...",1071944217,paypal
1685,2022-01-18,-12.99,"Lastschrift PAYPAL (EUROPE) S.A.R.L ET CIE,S.C...",1071944217,paypal
1686,2022-01-11,-228.09,"Lastschrift PAYPAL (EUROPE) S.A.R.L ET CIE,S.C...",1071944217,paypal
1687,2022-01-10,-89.96,"Lastschrift PAYPAL (EUROPE) S.A.R.L ET CIE,S.C...",1071944217,paypal


In [20]:
# df_dkb.to_csv('dkb.csv')

#### label by the most obvious keywords

In [21]:
categories

{0: 'Income - Salary & Benefits',
 1: 'Income - Presents',
 2: 'Income - Reimbursement',
 3: 'Income - Investments',
 4: 'Bill - Utilities, Rent & Fees',
 5: 'Bill - Communications',
 6: 'Bill - Insurrance',
 7: 'Bill - Health',
 8: 'Bill - Others',
 9: 'Holidays - Accomodation',
 10: 'Holidays - Transport',
 11: 'Holidays - Others',
 12: 'Car',
 13: 'Public Transport',
 14: 'Presents for Others',
 15: 'Supermarket & Everyday commodities',
 16: 'Restaurants & Bars',
 17: 'Entertainment, Hobbies & Memberships',
 18: 'Education',
 19: 'Personal Care & Sport',
 20: 'Investments',
 21: 'Other',
 22: 'Out-of-pocket expense',
 23: 'Home Improvement & Stationary',
 24: 'Clothes & Shopping'}

In [22]:
label_dict = {
    'Supermarket & Everyday commodities': ['Kartenzahlung/-abrechnung REWE', 'Kartenzahlung/-abrechnung EDEKA', 'Kartenzahlung/-abrechnung DIRK ROSSMANN', 'Kartenzahlung/-abrechnung DANKE, IHR LIDL', 'Kartenzahlung/-abrechnung DM', 'Kartenzahlung/-abrechnung MUELLER SAGT DANKE'],
    'Bill - Communications': ['Lastschrift Vodafone Deutschland GmbH'],
    'Bill - Utilities, Rent & Fees': ['Lastschrift Aberdeen Standard Investments'],
    'Income - Salary & Benefits': ['Lohn, Gehalt, Rente IAV GmbH', 'Lohn, Gehalt, Rente Hertie School', 'Gutschrift SavingBuddies e. V. Honorar'],
    'Income - Reimbursement': ['Gutschrift IAV GmbH'],
    'Public Transport': ['SWAPFIETS']
}

for label_desc, desc_list in label_dict.items():
    # label = {v:k for k, v in categories.items()}[label_desc]
    
    label_index = pd.DataFrame([df_dkb['desc'].str.contains(desc) for desc in desc_list]).T.any(axis=1)
    
    
    df_dkb.loc[label_index, 'class'] = label_desc # label
# df_dkb[df_dkb['desc'].str.contains('ROSSMANN')]

In [23]:
df_dkb[df_dkb['class'].isna() & df_dkb['target account'].isna()]

,date,amount,desc,account_nr,target account,class
9,2020-12-21,-6.90,Kartenzahlung/-abrechnung Lagard re Travel Ret...,1071944217,NaN,NaN
20,2020-12-14,-94.00,Kartenzahlung/-abrechnung BCRT REISEPRAXIS BER...,1071944217,NaN,NaN
21,2020-12-14,-30.60,Kartenzahlung/-abrechnung TEDI Fil 5995//Berli...,1071944217,NaN,NaN
23,2020-12-14,-85.00,Kartenzahlung/-abrechnung KARSTADT BEDANKT SIC...,1071944217,NaN,NaN
24,2020-12-14,-19.99,Kartenzahlung/-abrechnung Butlers GmbH + Co. K...,1071944217,NaN,NaN
...,...,...,...,...,...,...
1667,2022-02-01,-44.50,Kartenzahlung/-abrechnung BAUHAUS BERLIN-CHARL...,1071944217,NaN,NaN
1670,2022-01-31,-3.93,Kartenzahlung/-abrechnung BONG APOTHEKE//BERLI...,1071944217,NaN,NaN
1675,2022-01-24,-56.90,Überweisung DB VERTRIEB GMBH 100013522865 ...,1071944217,NaN,NaN
1681,2022-01-21,-21.95,Kartenzahlung/-abrechnung HUGENDUBEL 0665 FIL....,1071944217,NaN,NaN


In [24]:
df_dkb[df_dkb['desc'].str.contains('BVG')]

,date,amount,desc,account_nr,target account,class
13,2020-12-15,-9.0,"Lastschrift PAYPAL (EUROPE) S.A.R.L ET CIE,S.C...",1071944217,paypal,NaN
17,2020-12-15,-5.6,"Lastschrift PAYPAL (EUROPE) S.A.R.L ET CIE,S.C...",1071944217,paypal,NaN
28,2020-12-11,-5.6,"Lastschrift PAYPAL (EUROPE) S.A.R.L ET CIE,S.C...",1071944217,paypal,NaN
191,2020-08-25,-9.0,Kartenzahlung/-abrechnung BERLINER VERKEHRSBET...,1071944217,NaN,NaN
1087,2021-02-04,-65.0,Überweisung PAIGO GMBH BVG-EBE 310240217800446...,1071944217,NaN,NaN
1109,2021-01-19,-9.4,"Lastschrift PAYPAL (EUROPE) S.A.R.L ET CIE,S.C...",1071944217,paypal,NaN
1116,2021-01-14,-6.0,"Lastschrift PAYPAL (EUROPE) S.A.R.L ET CIE,S.C...",1071944217,paypal,NaN


#### save prelabeled data

In [25]:
df_dkb.to_csv('../data/dkb.csv')

In [26]:
df_dkb.loc[~paypal_indecies, 'amount'].sum()

125366.90000000001

In [50]:
'Überweisung HERR DR. JAN GIESELER'

'Überweisung HERR DR. JAN GIESELER'

In [49]:
df_dkb[df_dkb['amount']< -2000]

,date,amount,desc,account_nr
245,2021-12-01,-2270.93,Lastschrift Aberdeen Standard Investments Deut...,1002423109
246,2021-11-29,-20000.00,Lastschrift Sauren Select KAUF ISIN LU01155793...,1002423109
258,2021-11-01,-2270.93,Lastschrift Aberdeen Standard Investments Deut...,1002423109
274,2021-10-01,-2270.93,Lastschrift Aberdeen Standard Investments Deut...,1002423109
280,2021-09-10,-3000.00,Überweisung DKB VISACARD 4930000001554697 22.3...,1002423109
287,2021-09-01,-2270.93,Lastschrift Aberdeen Standard Investments Deut...,1002423109
303,2021-08-02,-2229.12,Lastschrift Aberdeen Standard Investments Deut...,1002423109
318,2021-07-01,-2229.12,Lastschrift Aberdeen Standard Investments Deut...,1002423109
329,2021-06-01,-2229.12,Lastschrift Aberdeen Standard Investments Deut...,1002423109
348,2021-05-03,-2229.12,Lastschrift Aberdeen Standard Investments Deut...,1002423109


## Playing around

### explore data

#### load the data

In [13]:
input_data_path =  Path('/Users/jangie/Documents/Finanzen/Detailed_Data')
account = '1002423109'

In [9]:
agg_data_file = Path('../all_data_dkb_1002423109.csv')

assert agg_data_file.exists()
df = pd.read_csv(str(agg_data_file))

In [ ]:
pp_raw


In [10]:
dkb_data_file = input_data_path/(f'dkb/2022_{account}.csv')

assert dkb_data_file.exists()
# df_dkb = pd.read_csv(str(dkb_data_file))
df_dkb = read_dkb_csv(dkb_data_file, verbose=2, drop_duplicates=True)
df_dkb

new data from /Users/jangie/Documents/Finanzen/Detailed_Data/dkb/2022_1002423109.csv with 148 entries, years [2022]
dropping 1 duplicated entries
147


,date,amount,desc,account_nr
0,2022-08-04,-40.00,Kartenzahlung/-abrechnung RESTAURANT SONNE GIR...,1002423109
2,2022-08-03,-100.00,Lastschrift European Bank for Financial Servic...,1002423109
3,2022-08-03,-23.00,Lastschrift GIRASOLES - SONNENBLUMEN Beitrag N...,1002423109
5,2022-08-02,-7.20,Kartenzahlung/-abrechnung Lagard re Travel Ret...,1002423109
7,2022-08-01,-2311.93,Lastschrift Aberdeen Standard Investments Deut...,1002423109
...,...,...,...,...
217,2022-01-07,-61.00,Lastschrift VATTENFALL EUROPE SALES S/83663436...,1002423109
219,2022-01-05,-100.00,Lastschrift European Bank for Financial Servic...,1002423109
221,2022-01-04,-2088.09,Lastschrift Aberdeen Standard Investments Deut...,1002423109
223,2022-01-03,-24.90,Lastschrift LVM Landw.Versicherungsverein Müns...,1002423109


In [11]:
# pd.to_datetime
# min_t = pd.to_datetime(df_dkb['date'].min())
# max_t = pd.to_datetime(df_dkb['date'].max())

In [12]:

# min_t = datetime.strftime(df_dkb['date'].min().to_pydatetime(), "%m/%d/%Y, %H:%M:%S")
# max_t = datetime.strftime(df_dkb['date'].max().to_pydatetime(), "%m/%d/%Y, %H:%M:%S")

In [27]:
pp_data_file = input_data_path/(f'Paypal/Paypal_01 Jan 2019 - 04 Aug 2022.csv')
assert pp_data_file.exists()
# df_dkb = pd.read_csv(str(dkb_data_file))
df_pp = read_paypal_csv(pp_data_file, verbose=2)
df_pp['Type'].value_counts()

loaded 1151 paypal records
890 paypal records after dropping `Balance Impact` = `Memo` entries
541 paypal records after dropping `Status` = `Pending` entries


Express Checkout Payment                 212
PreApproved Payment Bill User Payment    162
General Card Deposit                      59
General Payment                           35
Mobile Payment                            19
Payment Refund                            17
General Withdrawal                        13
General Currency Conversion               10
Reversal of General Account Hold           5
Donation Payment                           4
Website Payment                            2
Reversal of ACH Deposit                    1
Payment Reversal                           1
General Card Withdrawal                    1
Name: Type, dtype: Int64

In [14]:

min_t = datetime.strftime(df_dkb['date'].min().to_pydatetime(), "%m/%d/%Y, %H:%M:%S")
max_t = datetime.strftime(df_dkb['date'].max().to_pydatetime(), "%m/%d/%Y, %H:%M:%S")
min_t, max_t

('01/03/2022, 00:00:00', '08/04/2022, 00:00:00')

In [15]:
df_pp = df_pp[df_pp['datetime'].between(min_t,max_t, inclusive='both')]
len(df_pp), df_pp['datetime'].min(), df_pp['datetime'].max()

(174,
 Timestamp('2022-01-06 18:49:02+0000', tz='UTC'),
 Timestamp('2022-07-22 18:15:14+0000', tz='UTC'))

In [16]:
df_pp

,datetime,description,Date,Time,TimeZone,Name,Type,Status,Currency,amount,...,Address Line 2/District/Neighborhood,Town/City,State/Province/Region/County/Territory/Prefecture/Republic,Zip/Postal Code,Country,Contact Phone Number,Subject,Note,Country Code,Balance Impact
Transaction ID,,,,,,,,,,,,,,,,,,,,,
7UL384642K4863259-06012022 18:49:02+0000,2022-01-06 18:49:02+00:00,Airbnb Payments Luxembourg S.A. Acomodações n...,06/01/2022,19:49:02,+01:00,Airbnb Payments Luxembourg S.A.,PreApproved Payment Bill User Payment,Completed,EUR,-89.96,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Acomodações no Airbnb,<NA>,<NA>,Debit
9Y643689DP961835S-08012022 00:20:30+0000,2022-01-08 00:20:30+00:00,"Udacity, Inc Germany Machine Learning DevOps E...",08/01/2022,01:20:30,+01:00,"Udacity, Inc",PreApproved Payment Bill User Payment,Completed,BRL,-1389.00,...,22a,Berlin,<NA>,10585,Germany,<NA>,Machine Learning DevOps Engineer Nanodegree Pr...,<NA>,DE,Debit
0UD58843P8984980G-08012022 00:20:30+0000,2022-01-08 00:20:30+00:00,Machine Learning DevOps Engineer Nanodegree ...,08/01/2022,01:20:30,+01:00,<NA>,General Currency Conversion,Completed,EUR,-228.09,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Machine Learning DevOps Engineer Nanodegree Pr...,<NA>,<NA>,Debit
6JB317414D822921A-08012022 00:20:30+0000,2022-01-08 00:20:30+00:00,Machine Learning DevOps Engineer Nanodegree ...,08/01/2022,01:20:30,+01:00,<NA>,General Currency Conversion,Completed,BRL,1389.00,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Machine Learning DevOps Engineer Nanodegree Pr...,<NA>,<NA>,Credit
92V78974TU109551K-16012022 12:49:26+0000,2022-01-16 12:49:26+00:00,Spotify AB,16/01/2022,13:49:26,+01:00,Spotify AB,PreApproved Payment Bill User Payment,Completed,EUR,-12.99,...,<NA>,<NA>,<NA>,<NA>,<NA>,+49 15237806204,<NA>,<NA>,<NA>,Debit
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5W4962972W2662432-20072022 18:16:56+0000,2022-07-20 18:16:56+00:00,,20/07/2022,20:16:56,+02:00,<NA>,General Currency Conversion,Completed,EUR,-40.77,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Debit
71E76350SG496042D-20072022 18:16:56+0000,2022-07-20 18:16:56+00:00,,20/07/2022,20:16:56,+02:00,<NA>,General Currency Conversion,Completed,AUD,57.93,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Credit
7H59409744343521N-21072022 05:59:55+0000,2022-07-21 05:59:55+00:00,Shop der Deutschen Post (DPDHL Group),21/07/2022,07:59:55,+02:00,Shop der Deutschen Post (DPDHL Group),Express Checkout Payment,Completed,EUR,-1.00,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Debit


In [225]:
for s in df_pp['Status'].unique():
    print(s, len(df_pp['Status'][df_pp['Status']==s]))

Completed 174


In [226]:
for s in df_pp['Currency'].unique():
    print(s, len(df_pp['Currency'][df_pp['Currency']==s]))

EUR 170
BRL 2
AUD 2


In [240]:
for s in ['BRL', 'AUD']:
    print(s)
    print(df_pp[['datetime', 'amount', 'description']][df_pp['Currency']==s])

BRL
                                                          datetime  amount  \
Transaction ID                                                               
9Y643689DP961835S-08012022 00:20:30+0000 2022-01-08 00:20:30+00:00 -1389.0   
6JB317414D822921A-08012022 00:20:30+0000 2022-01-08 00:20:30+00:00  1389.0   

                                                                                description  
Transaction ID                                                                               
9Y643689DP961835S-08012022 00:20:30+0000  Udacity, Inc Germany Machine Learning DevOps E...  
6JB317414D822921A-08012022 00:20:30+0000    Machine Learning DevOps Engineer Nanodegree ...  
AUD
                                                          datetime  amount  \
Transaction ID                                                               
01652525P0036904F-20072022 18:16:56+0000 2022-07-20 18:16:56+00:00  -57.93   
71E76350SG496042D-20072022 18:16:56+0000 2022-07-20 18:16:56+00:00   

In [259]:
unique_datetimes = df_pp[df_pp['Currency']!='EUR']['datetime'].unique()
unique_datetimes[0]

Timestamp('2022-01-08 00:20:30+0000', tz='UTC')

In [270]:
df_pp[df_pp['Currency']!='EUR']

,datetime,description,Date,Time,TimeZone,Name,Type,Status,Currency,amount,...,Address Line 2/District/Neighborhood,Town/City,State/Province/Region/County/Territory/Prefecture/Republic,Zip/Postal Code,Country,Contact Phone Number,Subject,Note,Country Code,Balance Impact
Transaction ID,,,,,,,,,,,,,,,,,,,,,
9Y643689DP961835S-08012022 00:20:30+0000,2022-01-08 00:20:30+00:00,"Udacity, Inc Germany Machine Learning DevOps E...",08/01/2022,01:20:30,+01:00,"Udacity, Inc",PreApproved Payment Bill User Payment,Completed,BRL,-1389.00,...,22a,Berlin,<NA>,10585,Germany,<NA>,Machine Learning DevOps Engineer Nanodegree Pr...,<NA>,DE,Debit
6JB317414D822921A-08012022 00:20:30+0000,2022-01-08 00:20:30+00:00,Machine Learning DevOps Engineer Nanodegree ...,08/01/2022,01:20:30,+01:00,<NA>,General Currency Conversion,Completed,BRL,1389.00,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Machine Learning DevOps Engineer Nanodegree Pr...,<NA>,<NA>,Credit
01652525P0036904F-20072022 18:16:56+0000,2022-07-20 18:16:56+00:00,Discount Party Supplies Australia,20/07/2022,20:16:56,+02:00,Discount Party Supplies,Express Checkout Payment,Completed,AUD,-57.93,...,<NA>,Toorak,VIC,3142,Australia,<NA>,<NA>,<NA>,AU,Debit
71E76350SG496042D-20072022 18:16:56+0000,2022-07-20 18:16:56+00:00,,20/07/2022,20:16:56,+02:00,<NA>,General Currency Conversion,Completed,AUD,57.93,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Credit


In [268]:
''.join(df_pp[df_pp['datetime']==unique_datetimes[1]]['description'].unique())

'Discount Party Supplies Australia     '

In [272]:
def merge_foreign_transactions(df, verbose=1):
    """
    the description for foreign transactions is in the row with foreign currency
    here we find all the rows correcponding to the foreign transaction and fill in the description for the transaction in EUR
    
    """
    
    # the transactions are all recored at the same time, this is how we know which records belong together
    unique_datetimes = df[df['Currency']!='EUR']['datetime'].unique()
    if verbose>1:
        print(f'found {len(unique_datetimes)} foreign transactions')
        
    for unique_datetime in unique_datetimes:
        df_transaction = df[df['datetime']==unique_datetime]
        print(len(df_transaction))
        assert len(df_transaction[df_transaction['Currency']=='EUR']) == 1
        assert len(df_transaction[df_transaction['Currency']!='EUR']) == 2
        index = df_transaction[df_transaction['Currency']=='EUR'].index[0]
        print('index', index)
        # print(df_transaction[df_transaction['Currency']=='EUR'])[['Currency']]
        # replace the description in the EUR entry with the combined descriptions of all the entries
        ''.join(df_transaction['description'].unique())
        
    
    
    
merge_foreign_transactions(df_pp, verbose=2)
    

found 2 foreign transactions
3
index 0UD58843P8984980G-08012022 00:20:30+0000
3
index 5W4962972W2662432-20072022 18:16:56+0000


In [246]:
df_pp[df_pp['datetime'] == df_pp['datetime'][df_pp['Currency']==s][0]]

,datetime,description,Date,Time,TimeZone,Name,Type,Status,Currency,amount,...,Address Line 2/District/Neighborhood,Town/City,State/Province/Region/County/Territory/Prefecture/Republic,Zip/Postal Code,Country,Contact Phone Number,Subject,Note,Country Code,Balance Impact
Transaction ID,,,,,,,,,,,,,,,,,,,,,
01652525P0036904F-20072022 18:16:56+0000,2022-07-20 18:16:56+00:00,Discount Party Supplies Australia,20/07/2022,20:16:56,+02:00,Discount Party Supplies,Express Checkout Payment,Completed,AUD,-57.93,...,<NA>,Toorak,VIC,3142,Australia,<NA>,<NA>,<NA>,AU,Debit
5W4962972W2662432-20072022 18:16:56+0000,2022-07-20 18:16:56+00:00,,20/07/2022,20:16:56,+02:00,<NA>,General Currency Conversion,Completed,EUR,-40.77,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Debit
71E76350SG496042D-20072022 18:16:56+0000,2022-07-20 18:16:56+00:00,,20/07/2022,20:16:56,+02:00,<NA>,General Currency Conversion,Completed,AUD,57.93,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Credit


In [214]:
df_pp[df_pp['Status'] == 'Pending'][['amount', 'Balance', 'description']]

,amount,Balance,description
Transaction ID,,,
6L113162EB0103432-06012022 18:49:02+0000,89.96,0.00,Acomodações no Airbnb
4F5018144N338491K-08012022 00:20:30+0000,228.09,228.09,Machine Learning DevOps Engineer Nanodegree ...
7ER00513VT203380F-16012022 12:49:26+0000,12.99,0.00,
0RY33709RS715825G-17012022 04:32:08+0000,35.70,0.00,500k monthly pageviews
11R21029Y76470617-20012022 17:53:31+0000,5.00,0.00,CallYa
...,...,...,...
0R086892JX753043E-20072022 13:02:06+0000,22.50,0.00,Tageskarte (Mo-Fr); Boekingsreferentie 6835-1
23721860TE089781S-20072022 18:16:56+0000,40.77,40.77,
8VX5947311509125X-21072022 05:59:55+0000,1.00,0.00,


In [201]:
df_pp[df_pp['Status'] == 'Denied']

,datetime,description,Date,Time,TimeZone,Name,Type,Status,Currency,amount,...,Address Line 2/District/Neighborhood,Town/City,State/Province/Region/County/Territory/Prefecture/Republic,Zip/Postal Code,Country,Contact Phone Number,Subject,Note,Country Code,Balance Impact
Transaction ID,,,,,,,,,,,,,,,,,,,,,
0HW90013NX1945942-17042022 11:50:49+0000,2022-04-17 11:50:49+00:00,BVG Tickets 4-Fahrten-Karte-Regeltarif Berli...,17/04/2022,13:50:49,+02:00,<NA>,General Card Deposit,Denied,EUR,9.4,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,BVG Tickets 4-Fahrten-Karte-Regeltarif Berlin AB,<NA>,<NA>,Memo


In [194]:
df_pp[df_pp['Balance Impact'] == 'Credit']

,datetime,description,Date,Time,TimeZone,Name,Type,Status,Currency,amount,...,Address Line 2/District/Neighborhood,Town/City,State/Province/Region/County/Territory/Prefecture/Republic,Zip/Postal Code,Country,Contact Phone Number,Subject,Note,Country Code,Balance Impact
Transaction ID,,,,,,,,,,,,,,,,,,,,,
6L113162EB0103432-06012022 18:49:02+0000,2022-01-06 18:49:02+00:00,Acomodações no Airbnb,06/01/2022,19:49:02,+01:00,<NA>,Bank Deposit to PP Account,Pending,EUR,89.96,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Acomodações no Airbnb,<NA>,<NA>,Credit
4F5018144N338491K-08012022 00:20:30+0000,2022-01-08 00:20:30+00:00,Machine Learning DevOps Engineer Nanodegree ...,08/01/2022,01:20:30,+01:00,<NA>,Bank Deposit to PP Account,Pending,EUR,228.09,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Machine Learning DevOps Engineer Nanodegree Pr...,<NA>,<NA>,Credit
6JB317414D822921A-08012022 00:20:30+0000,2022-01-08 00:20:30+00:00,Machine Learning DevOps Engineer Nanodegree ...,08/01/2022,01:20:30,+01:00,<NA>,General Currency Conversion,Completed,BRL,1389.00,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Machine Learning DevOps Engineer Nanodegree Pr...,<NA>,<NA>,Credit
7ER00513VT203380F-16012022 12:49:26+0000,2022-01-16 12:49:26+00:00,,16/01/2022,13:49:26,+01:00,<NA>,Bank Deposit to PP Account,Pending,EUR,12.99,...,<NA>,<NA>,<NA>,<NA>,<NA>,+49 15237806204,<NA>,<NA>,<NA>,Credit
0RY33709RS715825G-17012022 04:32:08+0000,2022-01-17 04:32:08+00:00,500k monthly pageviews,17/01/2022,05:32:08,+01:00,<NA>,Bank Deposit to PP Account,Pending,EUR,35.70,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,500k monthly pageviews,<NA>,<NA>,Credit
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23721860TE089781S-20072022 18:16:56+0000,2022-07-20 18:16:56+00:00,,20/07/2022,20:16:56,+02:00,<NA>,Bank Deposit to PP Account,Pending,EUR,40.77,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Credit
71E76350SG496042D-20072022 18:16:56+0000,2022-07-20 18:16:56+00:00,,20/07/2022,20:16:56,+02:00,<NA>,General Currency Conversion,Completed,AUD,57.93,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Credit
8VX5947311509125X-21072022 05:59:55+0000,2022-07-21 05:59:55+00:00,,21/07/2022,07:59:55,+02:00,<NA>,Bank Deposit to PP Account,Pending,EUR,1.00,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Credit


In [110]:
# df_dkb[df_dkb['amount']==-7.9]

loaded 1151 paypal records
339 status completed


339

In [115]:
df_pp

,datetime,description,Type,amount,From Email Address,To Email Address
Transaction ID,,,,,,
22149102YJ0036701-26092020 06:53:18+0000,2020-09-26 06:53:18+00:00,Bestellung 10585-24705-37C7E9,Bank Deposit to PP Account,89.09,<NA>,jangie@pm.me
4N6325506F8414216-06102020 03:05:58+0000,2020-10-06 03:05:58+00:00,paypal-payment:6213322181486,Bank Deposit to PP Account,10.98,<NA>,jangie@pm.me
1XV032324M502644P-09102020 17:11:33+0000,2020-10-09 17:11:33+00:00,,Bank Deposit to PP Account,32.70,<NA>,jangie@pm.me
9NY88321ME4400257-09102020 19:17:41+0000,2020-10-09 19:17:41+00:00,CallYa,Bank Deposit to PP Account,10.00,<NA>,jangie@pm.me
0KL94453VJ859190X-15102020 10:24:14+0000,2020-10-15 10:24:14+00:00,CE 001500207,Bank Deposit to PP Account,127.68,<NA>,jangie@pm.me
...,...,...,...,...,...,...
0R086892JX753043E-20072022 13:02:06+0000,2022-07-20 13:02:06+00:00,Tageskarte (Mo-Fr); Boekingsreferentie 6835-1,Bank Deposit to PP Account,22.50,<NA>,jangie@pm.me
23721860TE089781S-20072022 18:16:56+0000,2022-07-20 18:16:56+00:00,,Bank Deposit to PP Account,40.77,<NA>,jangie@pm.me
8VX5947311509125X-21072022 05:59:55+0000,2022-07-21 05:59:55+00:00,,Bank Deposit to PP Account,1.00,<NA>,jangie@pm.me


In [15]:
df_pp['datetime'].min(), df_pp['datetime'].max()

(Timestamp('2020-09-26 06:53:18+0000', tz='UTC'),
 Timestamp('2022-07-22 18:15:14+0000', tz='UTC'))

In [18]:
df_dkb['date'].min(), df_dkb['date'].max()

(Timestamp('2022-01-03 00:00:00'), Timestamp('2022-08-04 00:00:00'))

In [28]:
df['date'].min(), df['date'].max()

('2020-01-03', '2022-08-04')

In [61]:
df_dkb[df_dkb['desc'].str.lower().str.contains('Lastschrift PAYPAL'.lower())]

,date,amount,desc
35,2022-07-05,-9.0,"Lastschrift PAYPAL (EUROPE) S.A.R.L ET CIE,S.C..."
75,2022-06-07,-15.0,"Lastschrift PAYPAL (EUROPE) S.A.R.L ET CIE,S.C..."


In [47]:
df_dkb[df_dkb['desc'].str.lower().str.contains('DKB'.lower())]

,date,amount,desc


#### find the paypal entries in the dkb data

In [30]:
df_dkb

,date,amount,desc,account_nr
0,2022-08-04,-40.00,Kartenzahlung/-abrechnung RESTAURANT SONNE GIR...,1002423109
2,2022-08-03,-100.00,Lastschrift European Bank for Financial Servic...,1002423109
3,2022-08-03,-23.00,Lastschrift GIRASOLES - SONNENBLUMEN Beitrag N...,1002423109
5,2022-08-02,-7.20,Kartenzahlung/-abrechnung Lagard re Travel Ret...,1002423109
7,2022-08-01,-2311.93,Lastschrift Aberdeen Standard Investments Deut...,1002423109
...,...,...,...,...
217,2022-01-07,-61.00,Lastschrift VATTENFALL EUROPE SALES S/83663436...,1002423109
219,2022-01-05,-100.00,Lastschrift European Bank for Financial Servic...,1002423109
221,2022-01-04,-2088.09,Lastschrift Aberdeen Standard Investments Deut...,1002423109
223,2022-01-03,-24.90,Lastschrift LVM Landw.Versicherungsverein Müns...,1002423109


In [37]:
paypal_index = get_paypal_index(df_dkb, df_pp, max_day_diff = 7, verbose=3)
paypal_index


 entry (amount -9.0): Lastschrift PAYPAL (EUROPE) S.A.R.L ET CIE,S.C.A. 1021012698462 PP.1384.PP
record: datetime                                                                              2022-07-04 06:11:49+00:00
description                                                   LogPay Financial Services GmbH  BVG Tickets 9-...
Date                                                                                                 04/07/2022
Time                                                                                                   08:11:49
TimeZone                                                                                                 +02:00
Name                                                                             LogPay Financial Services GmbH
Type                                                                      PreApproved Payment Bill User Payment
Status                                                                                                Complete

[(35, '14A60108TS633522P-04072022 06:11:49+0000'),
 (75, '9CJ43957AM7034146-03062022 14:54:00+0000')]

In [53]:
df_dkb

,date,amount,desc
0,2022-08-04,-40.00,Kartenzahlung/-abrechnung RESTAURANT SONNE GIR...
2,2022-08-03,-100.00,Lastschrift European Bank for Financial Servic...
3,2022-08-03,-23.00,Lastschrift GIRASOLES - SONNENBLUMEN Beitrag N...
5,2022-08-02,-7.20,Kartenzahlung/-abrechnung Lagard re Travel Ret...
7,2022-08-01,-2311.93,Lastschrift Aberdeen Standard Investments Deut...
...,...,...,...
217,2022-01-07,-61.00,Lastschrift VATTENFALL EUROPE SALES S/83663436...
219,2022-01-05,-100.00,Lastschrift European Bank for Financial Servic...
221,2022-01-04,-2088.09,Lastschrift Aberdeen Standard Investments Deut...
223,2022-01-03,-24.90,Lastschrift LVM Landw.Versicherungsverein Müns...


In [57]:
df_pp.iloc[35]

datetime                  2020-12-11 18:24:35+00:00
description             Salmon Lover Menü - normal 
Type                    Bank Deposit to PP Account 
amount                                        35.01
From Email Address                             <NA>
To Email Address                       jangie@pm.me
Name: 8VS55518BC890773C-11122020 18:24:35+0000, dtype: object

In [64]:
df_dkb.loc[[35]]

,date,amount,desc
35,2022-07-05,-9.0,"Lastschrift PAYPAL (EUROPE) S.A.R.L ET CIE,S.C..."


In [55]:
df_dkb.iloc[35]['target_account']= 'xx'

date                                    2022-06-28 00:00:00
amount                                               -30.85
desc      Kartenzahlung/-abrechnung REWE SAGT DANKE. 426...
Name: 53, dtype: object

In [26]:
df_pp['Type'].value_counts()

Express Checkout Payment                 71
PreApproved Payment Bill User Payment    64
General Card Deposit                     11
General Payment                          10
General Currency Conversion               4
Mobile Payment                            4
Payment Refund                            3
General Withdrawal                        3
Website Payment                           2
Reversal of General Account Hold          2
Name: Type, dtype: Int64

## paypal explore

In [17]:
paypal_file = '../data_extern/Paypal/Paypal_Activity_01 Jan 2019 - 17 Sep 2022.CSV'

In [18]:
pp_raw = pd.read_csv(paypal_file, decimal=",", dtype="string")
pp_raw['Gross'] = pp_raw['Gross'].str.replace('.','', regex=False).str.replace(',','.', regex=False).astype(float)
pp_raw['Type'].value_counts()

Bank Deposit to PP Account                361
General Authorization                     253
Express Checkout Payment                  226
PreApproved Payment Bill User Payment     169
General Card Deposit                       60
General Payment                            38
Mobile Payment                             22
General Withdrawal                         22
Payment Refund                             18
General Currency Conversion                12
Order                                       8
Void of Authorization                       6
Account Hold for Open Authorization         5
Reversal of General Account Hold            5
Donation Payment                            4
Reversal of ACH Deposit                     2
Website Payment                             2
Payment Reversal                            1
General Card Withdrawal                     1
Reversal of ACH Withdrawal Transaction      1
Name: Type, dtype: Int64

In [19]:
pp_raw[pp_raw['Gross'] == -0.01]

,Date,Time,TimeZone,Name,Type,Status,Currency,Gross,Fee,Net,...,Transaction Event Code,Payment Tracking ID,Bank Reference ID,Country Code,Balance Impact,Invoice Number.1,PO Number,Customer Reference Number,Payflow Transaction ID (PNREF),Original Invoice ID
354,24/04/2021,23:15:00,CEST,Spotify AB,General Authorization,Pending,EUR,-0.01,"0,00","-0,01",...,T1300,<NA>,<NA>,<NA>,Memo,<NA>,<NA>,<NA>,<NA>,<NA>
355,24/04/2021,23:15:03,CEST,Spotify AB,Void of Authorization,Reversed,EUR,-0.01,"0,00","-0,01",...,T1302,<NA>,<NA>,<NA>,Memo,<NA>,<NA>,<NA>,<NA>,<NA>
399,16/05/2021,14:49:19,CEST,Spotify AB,General Authorization,Pending,EUR,-0.01,"0,00","-0,01",...,T1300,<NA>,<NA>,<NA>,Memo,<NA>,<NA>,<NA>,<NA>,<NA>
400,16/05/2021,14:49:22,CEST,Spotify AB,Void of Authorization,Reversed,EUR,-0.01,"0,00","-0,01",...,T1302,<NA>,<NA>,<NA>,Memo,<NA>,<NA>,<NA>,<NA>,<NA>


In [89]:
pp_raw[['Type', 'Gross']].groupby('Type').agg('sum')

,Gross
Type,
Account Hold for Open Authorization,-153.44
Bank Deposit to PP Account,24958.23
Donation Payment,-53.00
Express Checkout Payment,-16799.30
General Authorization,-26776.94
General Card Deposit,1492.30
General Card Withdrawal,-36.85
General Currency Conversion,1727.35
General Payment,-540.97


### explore with facets

In [24]:

# Display the Dive visualization for the training data.
# from IPython.core.display import display, HTML
from IPython.display import display
from IPython.core.display import HTML

jsonstr = pp_raw.to_json(orient='records')
HTML_TEMPLATE = """
        
        
        
        """
html = HTML_TEMPLATE.format(jsonstr=jsonstr)
display(HTML(html))

## load all the data from several files

In [4]:
[f for f in (input_data_path/'dkb').glob('*.csv')]

NameError: name 'input_data_path' is not defined

In [ ]:
# ?pd.concat

In [453]:
df_dkb = None

for f in (input_data_path/'dkb').glob('*.csv'):
    assert f.exists()
    # df_dkb = pd.read_csv(str(dkb_data_file))
    df_dkb_in = read_dkb_csv(f, verbose=2, drop_duplicates=True)
    if df_dkb is None:
        df_dkb = df_dkb_in
    else: 
        df_dkb = pd.concat([df_dkb, df_dkb_in], ignore_index=True)
        print(f"dropping duplicates considering only ['date', 'amount', 'desc', 'account_nr']")
        df_dkb.drop_duplicates(subset=['date', 'amount', 'desc', 'account_nr'], inplace=True)

new data from /Users/jangie/Documents/Finanzen/Detailed_Data/dkb/2020_1071944217.csv with 232 entries, years [2020]
dropping 1 duplicated entries
231
new data from /Users/jangie/Documents/Finanzen/Detailed_Data/dkb/2021_1002423109.csv with 200 entries, years [2021]
dropping 0 duplicated entries
200
dropping duplicates considering only ['date', 'amount', 'desc', 'account_nr']
new data from /Users/jangie/Documents/Finanzen/Detailed_Data/dkb/2020_1002423109.csv with 135 entries, years [2020]
dropping 0 duplicated entries
135
dropping duplicates considering only ['date', 'amount', 'desc', 'account_nr']
new data from /Users/jangie/Documents/Finanzen/Detailed_Data/dkb/2021_1071944217.csv with 589 entries, years [2021]
dropping 4 duplicated entries
585
dropping duplicates considering only ['date', 'amount', 'desc', 'account_nr']
new data from /Users/jangie/Documents/Finanzen/Detailed_Data/dkb/2022_1002423109.csv with 148 entries, years [2022]
dropping 1 duplicated entries
147
dropping duplica

In [98]:
input_data_path

PosixPath('/Users/jangie/Documents/Finanzen/Detailed_Data')

In [445]:
[f for f in (input_data_path/'dkb').glob('*.csv')]

[PosixPath('/Users/jangie/Documents/Finanzen/Detailed_Data/dkb/2020_1071944217.csv'),
 PosixPath('/Users/jangie/Documents/Finanzen/Detailed_Data/dkb/2021_1002423109.csv'),
 PosixPath('/Users/jangie/Documents/Finanzen/Detailed_Data/dkb/2020_1002423109.csv'),
 PosixPath('/Users/jangie/Documents/Finanzen/Detailed_Data/dkb/2021_1071944217.csv'),
 PosixPath('/Users/jangie/Documents/Finanzen/Detailed_Data/dkb/2022_1002423109.csv'),
 PosixPath('/Users/jangie/Documents/Finanzen/Detailed_Data/dkb/2022_1071944217.csv')]

In [458]:
pp_data_file = input_data_path/(f'Paypal/Paypal_01 Jan 2019 - 29 Dec 2022.csv')
assert pp_data_file.exists()
# df_dkb = pd.read_csv(str(dkb_data_file))
df_pp = read_paypal_csv(pp_data_file, verbose=2)
df_pp['Type'].value_counts()

loaded 1382 paypal records
1084 paypal records after dropping `Balance Impact` = `Memo` entries


Bank Deposit to PP Account                426
Express Checkout Payment                  252
PreApproved Payment Bill User Payment     207
General Card Deposit                       59
General Payment                            39
Mobile Payment                             22
General Withdrawal                         21
Payment Refund                             21
General Currency Conversion                14
Account Hold for Open Authorization         5
Reversal of General Account Hold            5
Donation Payment                            4
Reversal of ACH Deposit                     3
Website Payment                             2
Reversal of ACH Withdrawal Transaction      2
Payment Reversal                            1
General Card Withdrawal                     1
Name: Type, dtype: Int64

In [459]:
# min_t = datetime.strftime(df_dkb['date'].min().to_pydatetime(), "%m/%d/%Y, %H:%M:%S")
# max_t = datetime.strftime(df_dkb['date'].max().to_pydatetime(), "%m/%d/%Y, %H:%M:%S")
min_t = datetime.strftime(df_dkb['date'].min().to_pydatetime(), "%d.%b.%Y")
max_t = datetime.strftime(df_dkb['date'].max().to_pydatetime(), "%d.%b.%Y")

print(f"dkb data exits from {min_t} to {max_t}")

dkb data exits from 03.Jan.2020 to 11.Aug.2022


In [460]:
min_t = datetime.strftime(df_pp['datetime'].min().to_pydatetime(), "%d.%b.%Y")
max_t = datetime.strftime(df_pp['datetime'].max().to_pydatetime(), "%d.%b.%Y")
print(f"paypal data exits from {min_t} to {max_t}")

paypal data exits from 27.May.2019 to 16.Dec.2022


#### find the paypal entries

In [475]:
def get_paypal_index_for_single_row(row, data_paypal, max_day_diff = 7, verbose=1):
    """
    row: datarow for which to find the paypal index, e.g. entry from dkb
    data_paypal: paypal data in which to look for the entry
    max_day_diff: max difference in days that the timestamp of the row may differ from the entry in the paypal data
    
    """

    # if  ('PAYPAL' not in row['desc'].upper()):
    #     return np.nan, None
    assert ('PAYPAL' in row['desc'].upper())
    
    # look for entries that have the same amount with opposite sign and have not been asigned yet
    paypal = data_paypal[(data_paypal['amount'] == -row['amount']) & (data_paypal['target'].isna())]
    # paypal = data_paypal[data_paypal['amount'] == -row['amount']]
    
    # data_paypal[data_paypal['Type'].str.strip().str.lower() == 'bank deposit to pp account']
    

    if len(paypal) == 0:
        if verbose>0:
            print(f"{row['date']} did not find entry with amount {row['amount']}")
        return np.nan, None

    record, time_diff =  get_closest_record(paypal, time_index = row['date'].tz_localize(timezone.utc))

    if abs(time_diff.days) > max_day_diff:
        if verbose>0:
            print(f"WARNING: large timediff {time_diff.days} for {len(paypal['datetime'])} records on {str(row['date']).split(' ')[0]} amount {row['amount']}")
            if verbose>1:
                print('***** paypal record ******')
                print(record)
                print('***** bank row ******')
                print(row['date'], row['amount'])
                print(row['desc'])
                print('\n')
        # return np.nan, None

    if verbose>2:
        print(f'\n entry (amount {row["amount"]}): {row["desc"]}')
        print('record:', record)
        
    data_paypal.loc[record.name, 'target'] = row.name
    return record.name, float(time_diff.days)


def get_paypal_index(data, data_paypal, max_day_diff = 7, verbose=1):
    """
    data: data for which to find the paypal index, e.g. entry from dkb
    data_paypal: paypal data in which to look for the entry
    max_day_diff: max difference in days that the timestamp of the row may differ from the entry in the paypal data
    
    returns list of tuples, first is the index in the data, second the  paypal index
    
    """
    indecies = []
    for i, row in data[data['desc'].str.lower().str.contains('paypal')].iterrows():
        index, days = get_paypal_index_for_single_row(row, data_paypal, max_day_diff = max_day_diff, verbose=verbose)
        indecies.append((i,index))
    return indecies


# get_paypal_index(df_dkb, df_pp, max_day_diff = 7, verbose=0)
# data = df_dkb
# max_day_diff = 7

data_paypal = df_pp
data_paypal['target']=np.nan
# verbose=1
c = 0
indecies = []
for i, row in data[data['desc'].str.lower().str.contains('lastschrift paypal')].iterrows():
    index, days = get_paypal_index_for_single_row(row, data_paypal, max_day_diff = max_day_diff, verbose=verbose)
    indecies.append((i,index))
#     c = c+1
#     if c>4:
        
#         break
    
# indecies

In [483]:
data_paypal[~data_paypal['target'].isna()]

,datetime,description,Date,Time,TimeZone,Name,Type,Status,Currency,amount,...,Payment Tracking ID,Bank Reference ID,Country Code,Balance Impact,Invoice Number.1,PO Number,Customer Reference Number,Payflow Transaction ID (PNREF),Original Invoice ID,target
Transaction ID,,,,,,,,,,,,,,,,,,,,,
22149102YJ0036701-26092020 06:53:18+0000,2020-09-26 06:53:18+00:00,Bestellung 10585-24705-37C7E9,26/09/2020,08:53:18,+02:00,<NA>,Bank Deposit to PP Account,Pending,EUR,89.09,...,<NA>,5CX23UU8FFZ7A,<NA>,Credit,<NA>,<NA>,<NA>,<NA>,<NA>,461.0
4N6325506F8414216-06102020 03:05:58+0000,2020-10-06 03:05:58+00:00,paypal-payment:6213322181486,06/10/2020,05:05:58,+02:00,<NA>,Bank Deposit to PP Account,Pending,EUR,10.98,...,<NA>,5CX23UUFUMEEC,<NA>,Credit,<NA>,<NA>,<NA>,<NA>,<NA>,476.0
1XV032324M502644P-09102020 17:11:33+0000,2020-10-09 17:11:33+00:00,,09/10/2020,19:11:33,+02:00,<NA>,Bank Deposit to PP Account,Pending,EUR,32.70,...,<NA>,5CX23UUJEBRLU,<NA>,Credit,<NA>,<NA>,<NA>,<NA>,<NA>,128.0
9NY88321ME4400257-09102020 19:17:41+0000,2020-10-09 19:17:41+00:00,CallYa,09/10/2020,21:17:41,+02:00,<NA>,Bank Deposit to PP Account,Pending,EUR,10.00,...,<NA>,5CX23UUJH2C3N,<NA>,Credit,<NA>,<NA>,<NA>,<NA>,<NA>,122.0
0KL94453VJ859190X-15102020 10:24:14+0000,2020-10-15 10:24:14+00:00,CE 001500207,15/10/2020,12:24:14,+02:00,<NA>,Bank Deposit to PP Account,Pending,EUR,127.68,...,<NA>,5CX23UUNC4SEY,<NA>,Credit,<NA>,<NA>,<NA>,<NA>,<NA>,472.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8VB40330L5065374J-21072022 16:29:58+0000,2022-07-21 16:29:58+00:00,Fritzing Backer,21/07/2022,18:29:58,+02:00,<NA>,Bank Deposit to PP Account,Pending,EUR,8.00,...,<NA>,1021346787361,<NA>,Credit,<NA>,<NA>,<NA>,<NA>,<NA>,1327.0
5E466442YM0945450-22072022 18:15:14+0000,2022-07-22 18:15:14+00:00,Salmon Lover Menü - normal,22/07/2022,20:15:14,+02:00,<NA>,Bank Deposit to PP Account,Pending,EUR,36.70,...,<NA>,1021366759723,<NA>,Credit,<NA>,<NA>,<NA>,<NA>,62dae8ada61f0,1318.0
93Y09253K1324434R-09082022 10:36:56+0000,2022-08-09 10:36:56+00:00,74774392,09/08/2022,12:36:56,+02:00,<NA>,Bank Deposit to PP Account,Pending,EUR,148.99,...,<NA>,1021685288504,<NA>,Credit,<NA>,<NA>,<NA>,<NA>,74774392,1289.0


In [474]:
data_paypal

,datetime,description,Date,Time,TimeZone,Name,Type,Status,Currency,amount,...,Payment Tracking ID,Bank Reference ID,Country Code,Balance Impact,Invoice Number.1,PO Number,Customer Reference Number,Payflow Transaction ID (PNREF),Original Invoice ID,target
Transaction ID,,,,,,,,,,,,,,,,,,,,,
4WY592183V7454844-27052019 07:02:28+0000,2019-05-27 07:02:28+00:00,Alexander Kihm Alles Liebe und viel Spass!! ...,27/05/2019,09:02:28,+02:00,Alexander Kihm,General Payment,Completed,EUR,-100.00,...,<NA>,<NA>,<NA>,Debit,<NA>,<NA>,<NA>,<NA>,<NA>,NaN
1CC51055UW323635G-03092020 08:31:34+0000,2020-09-03 08:31:34+00:00,Konrad Exner �,03/09/2020,10:31:34,+02:00,Konrad Exner,Mobile Payment,Completed,EUR,6.00,...,<NA>,<NA>,<NA>,Credit,<NA>,<NA>,<NA>,<NA>,<NA>,NaN
36F184282C430600K-26092020 06:53:18+0000,2020-09-26 06:53:18+00:00,DURSTEXPRESS GmbH Germany Bestellung 10585-247...,26/09/2020,08:53:18,+02:00,DURSTEXPRESS GmbH,Express Checkout Payment,Completed,EUR,-98.22,...,<NA>,<NA>,DE,Debit,<NA>,<NA>,<NA>,<NA>,<NA>,NaN
22149102YJ0036701-26092020 06:53:18+0000,2020-09-26 06:53:18+00:00,Bestellung 10585-24705-37C7E9,26/09/2020,08:53:18,+02:00,<NA>,Bank Deposit to PP Account,Pending,EUR,89.09,...,<NA>,5CX23UU8FFZ7A,<NA>,Credit,<NA>,<NA>,<NA>,<NA>,<NA>,NaN
9UY72313YA366071V-26092020 10:40:59+0000,2020-09-26 10:40:59+00:00,DURSTEXPRESS GmbH Bestellung 10585-24705-37C7E9,26/09/2020,12:40:59,+02:00,DURSTEXPRESS GmbH,Payment Refund,Completed,EUR,9.70,...,<NA>,<NA>,<NA>,Credit,<NA>,<NA>,<NA>,<NA>,<NA>,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8N691565N7322580S-10122022 12:16:36+0000,2022-12-10 12:16:36+00:00,Nike Order,10/12/2022,13:16:36,+01:00,<NA>,General Withdrawal,Completed,EUR,-33.97,...,<NA>,YYW1024024816919,<NA>,Debit,<NA>,<NA>,<NA>,<NA>,C01133505577,NaN
4TG0164811967221U-10122022 18:24:47+0000,2022-12-10 18:24:47+00:00,app smart GmbH / order smart Germany Salmon Lo...,10/12/2022,19:24:47,+01:00,app smart GmbH / order smart,Express Checkout Payment,Completed,EUR,-34.15,...,<NA>,<NA>,DE,Debit,<NA>,<NA>,<NA>,<NA>,6394ceb8a48fd,NaN
82B52026993747919-10122022 18:24:47+0000,2022-12-10 18:24:47+00:00,Salmon Lover Menü - normal,10/12/2022,19:24:47,+01:00,<NA>,Bank Deposit to PP Account,Pending,EUR,34.15,...,<NA>,1024030996502,<NA>,Credit,<NA>,<NA>,<NA>,<NA>,6394ceb8a48fd,NaN


In [464]:
data_paypal.loc[indecies[0][1]]['target']

0.0

In [482]:
df_dkb[df_dkb['desc'].str.upper().str.contains('PAYPAL')]

,date,amount,desc,account_nr
0,2020-12-29,-8.00,"Lastschrift PAYPAL (EUROPE) S.A.R.L ET CIE,S.C...",1071944217
1,2020-12-22,-9.98,"Lastschrift PAYPAL (EUROPE) S.A.R.L ET CIE,S.C...",1071944217
2,2020-12-22,-35.84,"Lastschrift PAYPAL (EUROPE) S.A.R.L ET CIE,S.C...",1071944217
3,2020-12-22,-33.39,"Lastschrift PAYPAL (EUROPE) S.A.R.L ET CIE,S.C...",1071944217
13,2020-12-15,-9.00,"Lastschrift PAYPAL (EUROPE) S.A.R.L ET CIE,S.C...",1071944217
...,...,...,...,...
1667,2022-01-24,-5.00,"Lastschrift PAYPAL (EUROPE) S.A.R.L ET CIE,S.C...",1071944217
1674,2022-01-18,-35.70,"Lastschrift PAYPAL (EUROPE) S.A.R.L ET CIE,S.C...",1071944217
1675,2022-01-18,-12.99,"Lastschrift PAYPAL (EUROPE) S.A.R.L ET CIE,S.C...",1071944217
1676,2022-01-11,-228.09,"Lastschrift PAYPAL (EUROPE) S.A.R.L ET CIE,S.C...",1071944217


In [485]:
data_paypal[~data_paypal['target'].isna()].count()

datetime                                                      333
description                                                   333
Date                                                          333
Time                                                          333
TimeZone                                                      333
Name                                                            3
Type                                                          333
Status                                                        333
Currency                                                      333
amount                                                        333
Fee                                                           333
Net                                                           333
From Email Address                                              2
To Email Address                                              333
Shipping Address                                                0
Address St

In [473]:
data_paypal[~data_paypal['target'].isna()]

,datetime,description,Date,Time,TimeZone,Name,Type,Status,Currency,amount,...,Payment Tracking ID,Bank Reference ID,Country Code,Balance Impact,Invoice Number.1,PO Number,Customer Reference Number,Payflow Transaction ID (PNREF),Original Invoice ID,target
Transaction ID,,,,,,,,,,,,,,,,,,,,,
30G99424U9197021X-11122020 16:07:55+0000,2020-12-11 16:07:55+00:00,BVG Tickets 4-Fahrten-Karte-Regeltarif Berli...,11/12/2020,17:07:55,+01:00,<NA>,Bank Deposit to PP Account,Pending,EUR,9.00,...,<NA>,5CX23UW3HVR6Q,<NA>,Credit,<NA>,<NA>,<NA>,<NA>,<NA>,13.0
649377949U676472L-18122020 17:10:29+0000,2020-12-18 17:10:29+00:00,Last But Not Least Menü (für 2 Personen) - n...,18/12/2020,18:10:29,+01:00,<NA>,Bank Deposit to PP Account,Pending,EUR,33.39,...,<NA>,5CX23UW9L798J,<NA>,Credit,<NA>,<NA>,<NA>,<NA>,<NA>,3.0
6DY89114A9480701J-20122020 07:57:38+0000,2020-12-20 07:57:38+00:00,,20/12/2020,08:57:38,+01:00,<NA>,Bank Deposit to PP Account,Pending,EUR,9.98,...,<NA>,5CX23UWAS3TMU,<NA>,Credit,<NA>,<NA>,<NA>,<NA>,<NA>,1.0
31N588582N0578938-20122020 22:06:51+0000,2020-12-20 22:06:51+00:00,Printastic Order,20/12/2020,23:06:51,+01:00,<NA>,Bank Deposit to PP Account,Pending,EUR,35.84,...,<NA>,5CX23UWB9SACQ,<NA>,Credit,<NA>,<NA>,<NA>,<NA>,<NA>,2.0
3RG54876WX339315M-26122020 19:15:09+0000,2020-12-26 19:15:09+00:00,Ihre Buchung bei bahn.de am 26.12.2020 mit d...,26/12/2020,20:15:09,+01:00,<NA>,Bank Deposit to PP Account,Pending,EUR,8.00,...,<NA>,5CX23UWF8B7L6,<NA>,Credit,<NA>,<NA>,<NA>,<NA>,<NA>,0.0


In [469]:
data_paypal[~data_paypal['target'].isna()]['amount'].count()

5

In [471]:
data_paypal[data_paypal['target'].isna()]['amount'].sum()

-196.44000000000057

In [417]:
data_paypal

,datetime,description,Date,Time,TimeZone,Name,Type,Status,Currency,amount,...,Town/City,State/Province/Region/County/Territory/Prefecture/Republic,Zip/Postal Code,Country,Contact Phone Number,Subject,Note,Country Code,Balance Impact,target
Transaction ID,,,,,,,,,,,,,,,,,,,,,
4WY592183V7454844-27052019 07:02:28+0000,2019-05-27 07:02:28+00:00,Alexander Kihm Alles Liebe und viel Spass!! ...,27/05/2019,09:02:28,+02:00,Alexander Kihm,General Payment,Completed,EUR,-100.00,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Alles Liebe und viel Spass!! Camila&Jan,<NA>,Debit,NaN
1CC51055UW323635G-03092020 08:31:34+0000,2020-09-03 08:31:34+00:00,Konrad Exner �,03/09/2020,10:31:34,+02:00,Konrad Exner,Mobile Payment,Completed,EUR,6.00,...,<NA>,<NA>,<NA>,<NA>,15204989713,<NA>,�,<NA>,Credit,NaN
36F184282C430600K-26092020 06:53:18+0000,2020-09-26 06:53:18+00:00,DURSTEXPRESS GmbH Germany Bestellung 10585-247...,26/09/2020,08:53:18,+02:00,DURSTEXPRESS GmbH,Express Checkout Payment,Completed,EUR,-98.22,...,Oberriexingen,<NA>,71739,Germany,<NA>,Bestellung 10585-24705-37C7E9,<NA>,DE,Debit,NaN
22149102YJ0036701-26092020 06:53:18+0000,2020-09-26 06:53:18+00:00,Bestellung 10585-24705-37C7E9,26/09/2020,08:53:18,+02:00,<NA>,Bank Deposit to PP Account,Pending,EUR,89.09,...,<NA>,<NA>,<NA>,<NA>,<NA>,Bestellung 10585-24705-37C7E9,<NA>,<NA>,Credit,NaN
9UY72313YA366071V-26092020 10:40:59+0000,2020-09-26 10:40:59+00:00,DURSTEXPRESS GmbH Bestellung 10585-24705-37C7E9,26/09/2020,12:40:59,+02:00,DURSTEXPRESS GmbH,Payment Refund,Completed,EUR,9.70,...,<NA>,<NA>,<NA>,<NA>,<NA>,Bestellung 10585-24705-37C7E9,<NA>,<NA>,Credit,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8VX5947311509125X-21072022 05:59:55+0000,2022-07-21 05:59:55+00:00,,21/07/2022,07:59:55,+02:00,<NA>,Bank Deposit to PP Account,Pending,EUR,1.00,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Credit,NaN
8MX23477622652713-21072022 16:29:58+0000,2022-07-21 16:29:58+00:00,Fritzing GmbH Fritzing Backer,21/07/2022,18:29:58,+02:00,Fritzing GmbH,Website Payment,Completed,EUR,-8.00,...,<NA>,<NA>,<NA>,<NA>,<NA>,Fritzing Backer,<NA>,<NA>,Debit,NaN
8VB40330L5065374J-21072022 16:29:58+0000,2022-07-21 16:29:58+00:00,Fritzing Backer,21/07/2022,18:29:58,+02:00,<NA>,Bank Deposit to PP Account,Pending,EUR,8.00,...,<NA>,<NA>,<NA>,<NA>,<NA>,Fritzing Backer,<NA>,<NA>,Credit,NaN


In [391]:
df_pp['target'] = np.nan
# df_pp.replace('<NA>', np.nan)


Transaction ID
4WY592183V7454844-27052019 07:02:28+0000    True
1CC51055UW323635G-03092020 08:31:34+0000    True
36F184282C430600K-26092020 06:53:18+0000    True
22149102YJ0036701-26092020 06:53:18+0000    True
9UY72313YA366071V-26092020 10:40:59+0000    True
                                            ... 
8N691565N7322580S-10122022 12:16:36+0000    True
4TG0164811967221U-10122022 18:24:47+0000    True
82B52026993747919-10122022 18:24:47+0000    True
2FL118868V267113H-16122022 12:49:28+0000    True
1VB43256WK5422840-16122022 12:49:28+0000    True
Name: target, Length: 1084, dtype: bool

,datetime,description,Date,Time,TimeZone,Name,Type,Status,Currency,amount,...,Payment Tracking ID,Bank Reference ID,Country Code,Balance Impact,Invoice Number.1,PO Number,Customer Reference Number,Payflow Transaction ID (PNREF),Original Invoice ID,target
Transaction ID,,,,,,,,,,,,,,,,,,,,,
9CN991136K372864L-19032022 09:12:35+0000,2022-03-19 09:12:35+00:00,Vodafone GmbH 15237806204-337858847-15EUR-PP-...,19/03/2022,10:12:35,+01:00,Vodafone GmbH,Express Checkout Payment,Completed,EUR,-15.0,...,<NA>,<NA>,<NA>,Debit,<NA>,<NA>,<NA>,<NA>,<NA>,NaN
39U5820294159400F-01042022 11:57:09+0000,2022-04-01 11:57:09+00:00,Deutsche Oper Berlin - Stiftung Oper in Berlin...,01/04/2022,13:57:09,+02:00,Deutsche Oper Berlin - Stiftung Oper in Berlin,Express Checkout Payment,Completed,EUR,-15.0,...,<NA>,<NA>,DE,Debit,<NA>,<NA>,<NA>,<NA>,<NA>,NaN
7KT94677W58059943-01042022 12:52:02+0000,2022-04-01 12:52:02+00:00,Deutsche Oper Berlin - Stiftung Oper in Berlin...,01/04/2022,14:52:02,+02:00,Deutsche Oper Berlin - Stiftung Oper in Berlin,Express Checkout Payment,Completed,EUR,-15.0,...,<NA>,<NA>,DE,Debit,<NA>,<NA>,<NA>,<NA>,<NA>,NaN
9CJ43957AM7034146-03062022 14:54:00+0000,2022-06-03 14:54:00+00:00,Vodafone GmbH 15237806204-337858847-15EUR-PP-...,03/06/2022,16:54:00,+02:00,Vodafone GmbH,Express Checkout Payment,Completed,EUR,-15.0,...,<NA>,<NA>,<NA>,Debit,<NA>,<NA>,<NA>,<NA>,<NA>,NaN
3R204914EH8560412-02072022 14:27:13+0000,2022-07-02 14:27:13+00:00,Vodafone GmbH 15237806204-337858847-15EUR-PP-...,02/07/2022,16:27:13,+02:00,Vodafone GmbH,Express Checkout Payment,Completed,EUR,-15.0,...,<NA>,<NA>,<NA>,Debit,<NA>,<NA>,<NA>,<NA>,<NA>,NaN
6FW80606XM803543T-09082022 13:10:01+0000,2022-08-09 13:10:01+00:00,Vodafone GmbH 15237806204-337858847-15EUR-PP-...,09/08/2022,15:10:01,+02:00,Vodafone GmbH,Express Checkout Payment,Completed,EUR,-15.0,...,<NA>,<NA>,<NA>,Debit,<NA>,<NA>,<NA>,<NA>,<NA>,NaN
3FH87895HW497550V-07102022 16:06:02+0000,2022-10-07 16:06:02+00:00,Curak Hana Camila's contribution to Svenja's...,07/10/2022,18:06:02,+02:00,Curak Hana,General Payment,Completed,EUR,-15.0,...,<NA>,<NA>,<NA>,Debit,<NA>,<NA>,<NA>,<NA>,<NA>,NaN


In [376]:
type(row)

pandas.core.series.Series

In [379]:
row.name

13

In [331]:
print('paypal data - number of deposits to pp account:', len(df_pp[df_pp['Type'].str.strip().str.lower() == 'bank deposit to pp account']))

print('dkb data - transfers to pp account:', len(df_dkb[df_dkb['desc'].str.lower().str.contains('paypal')]))

paypal data - number of deposits to pp account: 426
dkb data - transfers to pp account: 352


In [359]:
paypal_index = get_paypal_index(df_dkb, df_pp, max_day_diff = 100, verbose=1)

2020-10-07 00:00:00 did not find entry with amount |0.01|


In [362]:
counts = pd.Series(pp_index).value_counts(sort=False)
s_ambivalent = counts[counts>1]
print(f"found {len(s_ambivalent)} enties with ambivalent paypal entries")
s_ambivalent

found 13 enties with ambivalent paypal entries


4AV5054335637872N-11122020 14:17:12+0000    2
07F06641R1421130L-27112020 18:36:33+0000    3
1J3318263J477083J-26072021 20:20:15+0000    2
9VA60029WN1402340-19012021 20:15:48+0000    2
30Y895847U8930721-04072022 06:11:49+0000    3
4NN07388NV056110V-06072022 13:55:24+0000    2
5A888433D2029724R-14062022 12:33:00+0000    2
1CY20410NS861992B-25052022 12:56:48+0000    2
54115312LJ0303121-25052022 21:36:48+0000    2
46263748D9413491C-05052022 06:31:55+0000    2
96X94944S47156437-03052022 15:18:31+0000    2
2SP85967HB400905D-27042022 05:53:57+0000    2
7HE24813T42567035-06042022 06:08:20+0000    2
dtype: int64

In [367]:
df_pp.loc[s_ambivalent.index[0]]

datetime                                                                              2020-12-11 14:17:12+00:00
description                                                     BVG Tickets 4-Fahrten-Kurzstr.-Regeltarif Be...
Date                                                                                                 11/12/2020
Time                                                                                                   15:17:12
TimeZone                                                                                                 +01:00
Name                                                                                                       <NA>
Type                                                                                Bank Deposit to PP Account 
Status                                                                                                  Pending
Currency                                                                                                

In [343]:
pp_index  = [p for d, p in paypal_index]

s_unique = set(pp_index)
s_ambivalent = 

print(f"found {len(s_unique)} unique entries")

found 337 unique entries


In [344]:
.count('')

TypeError: list.count() takes exactly one argument (0 given)

In [255]:
i_dkb, i_pp = indecies[0]

print_infos_matched_entries(df_dkb, df_pp, *indecies[0])

dkb (2020-12-29 00:00:00):
	Lastschrift PAYPAL (EUROPE) S.A.R.L ET CIE,S.C.A. PP.9290.PP . DBVERTRIEBG, Ihr Einkauf bei DBVERTRIEBG
paypal (2020-12-26 19:15:09+00:00):
	  Ihre Buchung bei bahn.de am 26.12.2020 mit der Auftragsnummer HA78S7. 
	Bank Deposit to PP Account 


In [324]:
# df_pp.loc[i_pp]

352

2020-10-07 00:00:00 did not find entry with amount |0.01|


352

337

In [319]:
df_dkb[df_dkb['amount'] == -15]

,date,amount,desc,account_nr
30,2020-12-09,-15.0,Kartenzahlung/-abrechnung Azmaz und Islek GbR/...,1071944217
1192,2022-06-07,-15.0,"Lastschrift PAYPAL (EUROPE) S.A.R.L ET CIE,S.C...",1002423109
1290,2022-08-11,-15.0,"Lastschrift PAYPAL (EUROPE) S.A.R.L ET CIE,S.C...",1071944217
1374,2022-07-05,-15.0,"Lastschrift PAYPAL (EUROPE) S.A.R.L ET CIE,S.C...",1071944217
1534,2022-04-05,-15.0,"Lastschrift PAYPAL (EUROPE) S.A.R.L ET CIE,S.C...",1071944217
1563,2022-03-22,-15.0,"Lastschrift PAYPAL (EUROPE) S.A.R.L ET CIE,S.C...",1071944217


In [318]:
df_pp[df_pp['amount']== 15]

,datetime,description,Date,Time,TimeZone,Name,Type,Status,Currency,amount,...,Address Line 2/District/Neighborhood,Town/City,State/Province/Region/County/Territory/Prefecture/Republic,Zip/Postal Code,Country,Contact Phone Number,Subject,Note,Country Code,Balance Impact
Transaction ID,,,,,,,,,,,,,,,,,,,,,
7AJ09337FT611991E-19032022 09:12:35+0000,2022-03-19 09:12:35+00:00,CallYa,19/03/2022,10:12:35,+01:00,<NA>,Bank Deposit to PP Account,Pending,EUR,15.0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,CallYa,<NA>,<NA>,Credit
4FX854174X977930X-01042022 12:52:02+0000,2022-04-01 12:52:02+00:00,,01/04/2022,14:52:02,+02:00,<NA>,Bank Deposit to PP Account,Pending,EUR,15.0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Credit
4YR69494P28446230-03042022 09:35:14+0000,2022-04-03 09:35:14+00:00,Claudia Stedile Obrigada!!!,03/04/2022,11:35:14,+02:00,Claudia Stedile,Mobile Payment,Completed,EUR,15.0,...,<NA>,<NA>,<NA>,<NA>,<NA>,17663432421,<NA>,Obrigada!!!,<NA>,Credit
9LS40624K69149420-03062022 14:54:00+0000,2022-06-03 14:54:00+00:00,CallYa,03/06/2022,16:54:00,+02:00,<NA>,Bank Deposit to PP Account,Pending,EUR,15.0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,CallYa,<NA>,<NA>,Credit
7JJ81481KP289194R-02072022 14:27:13+0000,2022-07-02 14:27:13+00:00,CallYa,02/07/2022,16:27:13,+02:00,<NA>,Bank Deposit to PP Account,Pending,EUR,15.0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,CallYa,<NA>,<NA>,Credit


In [304]:
i=4
def print_infos_matched_entries(df_dkb, df_pp, i_dkb, i_pp):
    assert df_dkb.loc[i_dkb]['amount'] == -df_pp.loc[i_pp]['amount']
    print(f"dkb ({df_dkb.loc[i_dkb]['date']}):\n\t{df_dkb.loc[i_dkb]['desc']}")
    print(f"paypal ({df_pp.loc[i_pp]['datetime']}):\n\t{df_pp.loc[i_pp]['description']}\n\t{df_pp.loc[i_pp]['Type']}")
# print_infos_matched_entries(df_dkb, df_pp, *paypal_index[i])

In [135]:
for i, row in df_dkb.iterrows():
    'PAYPAL' in row['desc'].upper() 

In [162]:
paypal_index[59]

(602, '5MV32038UT735043K-22112021 07:41:34+0000')

### closer look at the entries that could not be found in paypal data

difference is due to different amounts in the dkb and paypal data, most likely because there is some credit in the paypal account left

In [283]:
[ i for i in paypal_index if isinstance(i[1], float) ]

[(132, nan), (1289, nan)]

In [287]:
i_dkb = 132
print(f"dkb ({df_dkb.loc[i_dkb]['date']}, {df_dkb.loc[i_dkb]['amount']}):\n\t{df_dkb.loc[i_dkb]['desc']}")

dkb (2020-10-07 00:00:00, 0.01):
	Gutschrift PayPal (Europe) S.a.r.l. et Cie., S.C.A. PP.9290.PP YYR1010631229099


In [290]:
df_dkb.loc[1289]

date                                        2022-08-11 00:00:00
amount                                                  -148.99
desc          Lastschrift PAYPAL (EUROPE) S.A.R.L ET CIE,S.C...
account_nr                                           1071944217
Name: 1289, dtype: object

In [202]:
for i, row in df_pp.iterrows():
    if ('IKEA' in row['description'].upper()):
        print(f"{row['datetime']}, {row['amount']}, {row['description']}")
        r2=row

2021-01-11 19:38:39+00:00, -74.94, IKEA Deutschland GmbH & Co. KG Germany IKEA 1158981106 
2021-11-06 21:39:12+00:00, -335.98, IKEA Deutschland GmbH & Co. KG Germany IKEA 1238177971 
2021-11-20 11:10:43+00:00, -98.93, IKEA Deutschland GmbH & Co. KG Germany IKEA 1241645612 
2022-03-02 20:16:10+00:00, -453.32, IKEA Deutschland GmbH & Co. KG Germany IKEA 1266884242 
2022-05-02 11:59:11+00:00, -138.96, IKEA Deutschland GmbH & Co. KG Germany IKEA 1279382030 


In [225]:
for i, row in pp_raw.fillna('').iterrows():
    if ('IKEA' in row['Item Title']):
        print(i, f"{row['Date']}, {row['Gross']}, {row['Item Title']}")
        r2=row

155 11/01/2021, -74.94, IKEA 1158981106
156 11/01/2021, 74.94, IKEA 1158981106
701 06/11/2021, -335.98, IKEA 1238177971
702 06/11/2021, 335.98, IKEA 1238177971
729 20/11/2021, -98.93, IKEA 1241645612
730 20/11/2021, 88.9, IKEA 1241645612
865 02/03/2022, -453.32, IKEA 1266884242
866 02/03/2022, 453.32, IKEA 1266884242
980 02/05/2022, -138.96, IKEA 1279382030
981 02/05/2022, 138.96, IKEA 1279382030


In [229]:
pp_raw.loc[729]

Date                                                                                  20/11/2021
Time                                                                                    12:10:43
TimeZone                                                                                     CET
Name                                                              IKEA Deutschland GmbH & Co. KG
Type                                                                    Express Checkout Payment
Status                                                                                 Completed
Currency                                                                                     EUR
Gross                                                                                     -98.93
Fee                                                                                         0,00
Net                                                                                       -98,93
From Email Address            

In [230]:
pp_raw.loc[730]

Date                                                                           20/11/2021
Time                                                                             12:10:43
TimeZone                                                                              CET
Name                                                                                 <NA>
Type                                                          Bank Deposit to PP Account 
Status                                                                            Pending
Currency                                                                              EUR
Gross                                                                                88.9
Fee                                                                                  0,00
Net                                                                                 88,90
From Email Address                                                                   <NA>
To Email A

In [211]:
pp_raw.loc[100]

Date                                                                                                09/12/2020
Time                                                                                                  16:03:47
TimeZone                                                                                                   CET
Name                                                                                                      <NA>
Type                                                                               Bank Deposit to PP Account 
Status                                                                                                 Pending
Currency                                                                                                   EUR
Gross                                                                                                      5.6
Fee                                                                                                       0,00
N

In [109]:
df_pp.loc[paypal_index[i][1]]

datetime                                                                              2020-12-26 19:15:09+00:00
description                                                   DB Vertrieb GmbH  Ihre Buchung bei bahn.de am ...
Date                                                                                                 26/12/2020
Time                                                                                                   20:15:09
TimeZone                                                                                                 +01:00
Name                                                                                           DB Vertrieb GmbH
Type                                                                                   Express Checkout Payment
Status                                                                                                Completed
Currency                                                                                                

In [204]:
df_dkb

,date,amount,desc,account_nr
0,2020-12-29,-8.00,"Lastschrift PAYPAL (EUROPE) S.A.R.L ET CIE,S.C...",1071944217
1,2020-12-22,-9.98,"Lastschrift PAYPAL (EUROPE) S.A.R.L ET CIE,S.C...",1071944217
2,2020-12-22,-35.84,"Lastschrift PAYPAL (EUROPE) S.A.R.L ET CIE,S.C...",1071944217
3,2020-12-22,-33.39,"Lastschrift PAYPAL (EUROPE) S.A.R.L ET CIE,S.C...",1071944217
4,2020-12-21,-5.37,Kartenzahlung/-abrechnung REWE SAGT DANKE. 426...,1071944217
...,...,...,...,...
1674,2022-01-18,-35.70,"Lastschrift PAYPAL (EUROPE) S.A.R.L ET CIE,S.C...",1071944217
1675,2022-01-18,-12.99,"Lastschrift PAYPAL (EUROPE) S.A.R.L ET CIE,S.C...",1071944217
1676,2022-01-11,-228.09,"Lastschrift PAYPAL (EUROPE) S.A.R.L ET CIE,S.C...",1071944217
1677,2022-01-10,-89.96,"Lastschrift PAYPAL (EUROPE) S.A.R.L ET CIE,S.C...",1071944217


,Date,Time,TimeZone,Name,Type,Status,Currency,Gross,Fee,Net,...,Transaction Event Code,Payment Tracking ID,Bank Reference ID,Country Code,Balance Impact,Invoice Number.1,PO Number,Customer Reference Number,Payflow Transaction ID (PNREF),Original Invoice ID
0,27/05/2019,09:02:28,CEST,Alexander Kihm,General Payment,Completed,EUR,-100.00,"0,00","-100,00",...,T0000,<NA>,<NA>,<NA>,Debit,<NA>,<NA>,<NA>,<NA>,<NA>
1,03/09/2020,10:31:34,CEST,Konrad Exner,Mobile Payment,Completed,EUR,6.00,"0,00","6,00",...,T0011,<NA>,<NA>,<NA>,Credit,<NA>,<NA>,<NA>,<NA>,<NA>
2,26/09/2020,08:53:18,CEST,DURSTEXPRESS GmbH,Express Checkout Payment,Completed,EUR,-98.22,"0,00","-98,22",...,T0006,<NA>,<NA>,DE,Debit,<NA>,<NA>,<NA>,<NA>,<NA>
3,26/09/2020,08:53:18,CEST,<NA>,Bank Deposit to PP Account,Pending,EUR,89.09,"0,00","89,09",...,T0300,<NA>,5CX23UU8FFZ7A,<NA>,Credit,<NA>,<NA>,<NA>,<NA>,<NA>
4,26/09/2020,12:40:59,CEST,DURSTEXPRESS GmbH,Payment Refund,Completed,EUR,9.70,"0,00","9,70",...,T1107,<NA>,<NA>,<NA>,Credit,<NA>,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1211,09/09/2022,16:22:02,CEST,<NA>,Bank Deposit to PP Account,Pending,EUR,5.00,"0,00","5,00",...,T0300,<NA>,1022251969436,<NA>,Credit,<NA>,<NA>,<NA>,<NA>,PP22875889
1212,09/09/2022,16:22:53,CEST,Vodafone GmbH,Express Checkout Payment,Completed,EUR,-5.00,"0,00","-5,00",...,T0006,<NA>,<NA>,<NA>,Debit,<NA>,<NA>,<NA>,<NA>,<NA>
1213,09/09/2022,16:22:53,CEST,<NA>,Bank Deposit to PP Account,Pending,EUR,5.00,"0,00","5,00",...,T0300,<NA>,1022251983553,<NA>,Credit,<NA>,<NA>,<NA>,<NA>,PP22875897
1214,16/09/2022,14:49:27,CEST,Spotify AB,PreApproved Payment Bill User Payment,Completed,EUR,-12.99,"0,00","-12,99",...,T0003,<NA>,<NA>,<NA>,Debit,<NA>,<NA>,<NA>,<NA>,<NA>


### expore and clean dkb data

In [323]:
df_dkb['']

,date,amount,desc,account_nr
0,2020-12-29,-8.00,"Lastschrift PAYPAL (EUROPE) S.A.R.L ET CIE,S.C...",1071944217
1,2020-12-22,-9.98,"Lastschrift PAYPAL (EUROPE) S.A.R.L ET CIE,S.C...",1071944217
2,2020-12-22,-35.84,"Lastschrift PAYPAL (EUROPE) S.A.R.L ET CIE,S.C...",1071944217
3,2020-12-22,-33.39,"Lastschrift PAYPAL (EUROPE) S.A.R.L ET CIE,S.C...",1071944217
4,2020-12-21,-5.37,Kartenzahlung/-abrechnung REWE SAGT DANKE. 426...,1071944217
...,...,...,...,...
1674,2022-01-18,-35.70,"Lastschrift PAYPAL (EUROPE) S.A.R.L ET CIE,S.C...",1071944217
1675,2022-01-18,-12.99,"Lastschrift PAYPAL (EUROPE) S.A.R.L ET CIE,S.C...",1071944217
1676,2022-01-11,-228.09,"Lastschrift PAYPAL (EUROPE) S.A.R.L ET CIE,S.C...",1071944217
1677,2022-01-10,-89.96,"Lastschrift PAYPAL (EUROPE) S.A.R.L ET CIE,S.C...",1071944217
